In [53]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()

roi = config.roi
data_folder = config.data_folder

biomass_raw = ee.Image(f"{data_folder}/raw/biomass")
proj = biomass_raw.projection().getInfo()
method = 'eu'

panama = False
if panama:
    roi = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filterMetadata("country_na", "equals", "Panama").geometry()
    folder = "panama"
else:
    folder = method

In [55]:
mature_biomass = ee.Image(f"{data_folder}/{method}/mature_biomass_500")
mature_biomass_vector = ee.FeatureCollection(f"{data_folder}/{folder}/mature_biomass_vector")
distance_to_nearest_mature = ee.Image("projects/amazon-forest-regrowth/assets/mapbiomas/distance_to_nearest_mature")
one_hectare_mask = ee.Image(f"{data_folder}/{method}/one_hectare_mask")
secondary = ee.Image(f"{data_folder}/{method}/secondary")
secondary = secondary.select("age").updateMask(one_hectare_mask)

def buffer_distance(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(1000)))
    return buffered

# Load region of interest (ROI) and create a grid over the ROI
grid = roi.coveringGrid("EPSG:4326", 1000000)
tile_ids = grid.aggregate_array('system:index').getInfo()
count = 0

# Loop over IDs
for feature_id in tile_ids:
    count = count + 1

    # Find the feature again by its ID in the collection.
    feat = grid.filter(ee.Filter.eq('system:index', feature_id))

    distances_vector = distance_to_nearest_mature.sample(region = feat.geometry(), scale = 1000, geometries = True)

    distances_vector_buffer = distances_vector.map(buffer_distance)

    nearest_mature_biomass = mature_biomass.reduceRegions(distances_vector_buffer, ee.Reducer.firstNonNull(), 
                                            crs = proj['crs'], crsTransform = proj['transform'])

    # change geometries from polygons to points
    nearest_mature_biomass = nearest_mature_biomass.map(lambda f: ee.Feature(f.geometry().centroid()).copyProperties(f))

    # Convert the FeatureCollection to an image
    nearest_mature_biomass_image = nearest_mature_biomass.reduceToImage(
        properties = ['first'],
        reducer = ee.Reducer.first()
    ).toInt16().rename('nearest_mature_biomass')

    export_image(nearest_mature_biomass_image, f"nearest_mature_biomass_{count}", \
                    region = feat.geometry(), folder = method, scale = 1000)

In [50]:
base_name = "projects/amazon-forest-regrowth/assets/mapbiomas/nearest_mature_biomass_"
suffixes = range(1, 8)
images = [ee.Image(f"{base_name}{suffix}") for suffix in suffixes]
image_collection = ee.ImageCollection(images)
merged_image = image_collection.mosaic()
merged_image = merged_image.reproject(scale = 1000)
tst = ee.Image("projects/amazon-forest-regrowth/assets/mapbiomas/nearest_mature_biomass_image")

map = geemap.Map()
map.addLayer(mature_biomass_vector, {}, 'mature_biomass_vector')
map.addLayer(merged_image, {'min': 0, 'max': 400, 'palette': ['red', 'blue']}, 'nearest mature biomass')
# map.addLayer(mature_biomass, {'min': 0, 'max': 400, 'palette': ['red', 'blue']}, 'mature biomass')
map.addLayer(distance_to_nearest_mature, {}, 'distance_to_nearest_mature', opacity = 0.7)
map.addLayer(secondary, {}, 'secondary', opacity = 0.7)
map.addLayer(tst, {'min': 0, 'max': 400, 'palette': ['red', 'blue']}, 'tst')
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [22]:

gaus_kernel = ee.Kernel.gaussian(radius = 10000, sigma = 200, units = "meters")

mat_gaus_ker = (
    mature_biomass.focalMean(kernel = gaus_kernel, iterations = 1)
    .rename("mat_biomass")
    .float()
)

export_image(mat_gaus_ker, "mat_gaus_ker", region = roi, folder = method, crsTransform = proj['transform'])
